In [4]:
from pathlib import Path
import pandas as pd
import random
import soundfile as sf
import numpy as np

# FG 音频根目录（你之前已经填过的话保持不变）
FG_WAV_ROOT = Path(r"X:\数据集\OceanShip\OceanShip-FG")   # 示例，请改成你真实 FG 路径
CG_WAV_ROOT = Path(r"X:\数据集\OceanShip\OceanShip-CG")   # 你的 CG wav 根目录

# FG 标注文件（GitHub 上的那两个）
FG_META_FILES = [
    Path(r"X:\数据集\OceanShip\oceanship_fg_train.csv"),
    Path(r"X:\数据集\OceanShip\oceanship_fg_test.csv"),
]

# CG 标注文件——现在就只有这一个
CG_META_FILES = [
    Path(r"X:\数据集\OceanShip\OceanShip-CG\label.csv"),
]

In [9]:
def load_expected_basenames(meta_files, wav_col_candidates=None):
    """
    从若干个标注文件中，抽取“期望存在的 wav 文件名集合”
    """
    if wav_col_candidates is None:
        # 把可能的列名多放几个，兼容 CG 的 label.csv
        wav_col_candidates = [
            "wav_path", "file", "filename", "wav", "path",
            "audio", "audio_path", "audio_file", "name"
        ]

    dfs = []
    used_col = None

    for meta_path in meta_files:
        df = pd.read_csv(meta_path)
        print(f"\n[INFO] 读取标注文件: {meta_path}")
        print("[INFO] 列名：", list(df.columns))

        # 自动探测哪一列是 wav 路径
        col = None
        for c in wav_col_candidates:
            if c in df.columns:
                col = c
                break
        if col is None:
            raise ValueError(
                f"在标注文件 {meta_path} 中找不到 wav 路径列，"
                f"请查看该 csv 的列名，并在 wav_col_candidates 中加上正确列名。"
            )
        if used_col is None:
            used_col = col
        print(f"[INFO] 使用列 '{col}' 作为 wav 路径")

        sub = df[[col]].copy()
        sub["basename"] = sub[col].astype(str).apply(lambda p: Path(p).name)
        dfs.append(sub[["basename"]])

    all_df = pd.concat(dfs, ignore_index=True)

    # 查重
    dup_counts = all_df["basename"].value_counts()
    duplicates = dup_counts[dup_counts > 1]

    expected = set(all_df["basename"])
    print(f"\n[INFO] 标注中共出现 {len(expected)} 个唯一 wav 文件名，总行数 {len(all_df)}，"
          f"其中重复文件名 {len(duplicates)} 个")

    return expected, duplicates


def scan_wav_basenames(wav_root: Path):
    """
    扫描目录下所有 wav，返回 basenames 集合和路径列表
    """
    wav_paths = list(wav_root.rglob("*.wav"))
    basenames = {p.name for p in wav_paths}
    print(f"[INFO] {wav_root} 下扫描到 {len(wav_paths)} 个 wav 文件，"
          f"其中 {len(basenames)} 个唯一文件名")
    return basenames, wav_paths


In [12]:
def load_expected_stems(meta_files, wav_col="wav_path"):
    stems = []
    for meta_path in meta_files:
        df = pd.read_csv(meta_path)
        s = df[wav_col].astype(str).apply(lambda p: Path(p).stem)
        stems.append(s)
        print(f"[INFO] {meta_path} 使用列 '{wav_col}'，共 {len(s)} 条")
    all_stems = pd.concat(stems, ignore_index=True)
    unique_stems = set(all_stems.tolist())
    return unique_stems

def scan_wav_stems(wav_root: Path):
    wav_paths = list(wav_root.rglob("*.wav"))
    stems = {p.stem for p in wav_paths}
    print(f"[INFO] {wav_root} 下扫描到 {len(wav_paths)} 个 wav，{len(stems)} 个唯一 stem")
    return stems, wav_paths

# FG 检查
fg_expected_stems = load_expected_stems(FG_META_FILES, wav_col="wav_path")
fg_actual_stems, fg_paths = scan_wav_stems(FG_WAV_ROOT)

fg_missing_stems = sorted(fg_expected_stems - fg_actual_stems)
fg_extra_stems   = sorted(fg_actual_stems - fg_expected_stems)

print(f"[FG] 期望 stem 数量: {len(fg_expected_stems)}")
print(f"[FG] 实际 stem 数量: {len(fg_actual_stems)}")
print(f"[FG] 缺失 stem: {len(fg_missing_stems)}")
print(f"[FG] 多余 stem: {len(fg_extra_stems)}")
print("前 10 个缺失 stem:", fg_missing_stems[:10])
print("前 10 个多余 stem:", fg_extra_stems[:10])

[INFO] X:\数据集\OceanShip\oceanship_fg_train.csv 使用列 'wav_path'，共 45699 条
[INFO] X:\数据集\OceanShip\oceanship_fg_test.csv 使用列 'wav_path'，共 8071 条
[INFO] X:\数据集\OceanShip\OceanShip-FG 下扫描到 53770 个 wav，53770 个唯一 stem
[FG] 期望 stem 数量: 53770
[FG] 实际 stem 数量: 53770
[FG] 缺失 stem: 0
[FG] 多余 stem: 0
前 10 个缺失 stem: []
前 10 个多余 stem: []


In [13]:
def load_expected_stems(meta_files, wav_col):
    """
    从若干个标注文件中，抽取“期望存在的 wav 文件名 stem 集合”
    wav_col: 标注文件中表示音频路径/文件名的列名，比如 'wav_path' 或 'path'
    """
    stems = []
    for meta_path in meta_files:
        df = pd.read_csv(meta_path)
        print(f"[INFO] 读取标注文件: {meta_path}")
        print("[INFO] 列名：", list(df.columns))
        if wav_col not in df.columns:
            raise ValueError(f"{meta_path} 中找不到列 '{wav_col}'，请检查列名。")
        s = df[wav_col].astype(str).apply(lambda p: Path(p).stem)
        stems.append(s)
        print(f"[INFO] 使用列 '{wav_col}'，共 {len(s)} 条")

    all_stems = pd.concat(stems, ignore_index=True)
    unique_stems = set(all_stems.tolist())
    print(f"[INFO] 标注中共出现 {len(unique_stems)} 个唯一 stem")
    return unique_stems


def scan_wav_stems(wav_root: Path):
    """
    扫描目录下所有 wav，返回 stem 集合和完整路径列表
    """
    wav_paths = list(wav_root.rglob("*.wav"))
    stems = {p.stem for p in wav_paths}
    print(f"[INFO] {wav_root} 下扫描到 {len(wav_paths)} 个 wav，{len(stems)} 个唯一 stem")
    return stems, wav_paths

# 从 label.csv 中读出期望存在的 stem
cg_expected_stems = load_expected_stems(
    CG_META_FILES,
    wav_col="path"    # 这里指定使用 label.csv 的 'path' 列
)

# 扫描目录里的 .wav，取 stem
cg_actual_stems, cg_paths = scan_wav_stems(CG_WAV_ROOT)

# 差集比较
cg_missing = sorted(cg_expected_stems - cg_actual_stems)  # 标注有，目录没有
cg_extra   = sorted(cg_actual_stems - cg_expected_stems)  # 目录有，标注没有

print(f"\n[CG] 标注中期望的 stem 数量: {len(cg_expected_stems)}")
print(f"[CG] 实际 stem 数量: {len(cg_actual_stems)}")
print(f"[CG] 缺失 stem: {len(cg_missing)}")
print(f"[CG] 多余 stem: {len(cg_extra)}\n")

print("前 10 个缺失 stem：", cg_missing[:10])
print("前 10 个多余 stem：", cg_extra[:10])


[INFO] 读取标注文件: X:\数据集\OceanShip\OceanShip-CG\label.csv
[INFO] 列名： ['label', 'path']
[INFO] 使用列 'path'，共 49153 条
[INFO] 标注中共出现 49153 个唯一 stem
[INFO] X:\数据集\OceanShip\OceanShip-CG 下扫描到 35671 个 wav，35671 个唯一 stem

[CG] 标注中期望的 stem 数量: 49153
[CG] 实际 stem 数量: 35671
[CG] 缺失 stem: 13482
[CG] 多余 stem: 0

前 10 个缺失 stem： ['20210115T113626.931Z_1380_id_5_typecargo_70', '20210115T113810.930Z_1383_id_5_typecargo_52', '20210115T113859.095Z_1385_id_24_typecargo_37_0', '20210115T113859.095Z_1385_id_24_typecargo_37_1', '20210115T114056.210Z_1390_id_5_typecargo_70_0', '20210115T114210.137Z_1393_id_24_typecargo_37_0', '20210115T114224.584Z_1394_id_5_typecargo_70', '20210115T114252.511Z_1395_id_24_typecargo_37', '20210115T114336.664Z_1397_id_5_typecargo_31', '20210115T114403.151Z_1398_id_24_typecargo_30']
前 10 个多余 stem： []


# Train文件夹标注去重处理

In [ ]:
# 1. 配置路径：根据你本地情况修改
META_CSV = Path(r"X:\数据集\OceanShip_2\oceanship_full_train.csv")
WAV_ROOT = Path(r"X:\数据集\OceanShip_2\Oceanship_train")

WAV_COL_NAME = "wav_path"  # 标注中表示音频路径的列名

# 2. 读取标注，统计行数和唯一 stem 数
df = pd.read_csv(META_CSV)
print("标注列名：", list(df.columns))
print("原标注行数：", len(df))

# 取 stem（去掉目录和后缀）
df["stem"] = df[WAV_COL_NAME].astype(str).apply(lambda p: Path(p).stem)

expected_stems = df["stem"]
expected_stem_set = set(expected_stems.tolist())
print("标注中的唯一 stem 数量：", len(expected_stem_set))

# 3. 扫描 train 目录下所有 wav，统计实际 stem 集合
wav_paths = list(WAV_ROOT.rglob("*.wav"))
actual_stem_set = {p.stem for p in wav_paths}

print(f"\n{WAV_ROOT} 下共找到 {len(wav_paths)} 个 wav 文件，"
      f"{len(actual_stem_set)} 个唯一 stem。")

# 4. 计算差集：缺失 / 多余
missing_stems = sorted(expected_stem_set - actual_stem_set)  # 标注有，目录没有
extra_stems   = sorted(actual_stem_set - expected_stem_set)  # 目录有，标注没有

print("\n[Train 检查结果]")
print(f"标注期望的 stem 数量: {len(expected_stem_set)}")
print(f"实际 wav stem 数量: {len(actual_stem_set)}")
print(f"缺失的 wav 数量: {len(missing_stems)}")
print(f"多余的 wav 数量: {len(extra_stems)}")

print("\n前 10 个缺失的 stem：", missing_stems[:10])
print("前 10 个多余的 stem：", extra_stems[:10])

标注列名： ['wav_path', 'label', 'text_path']
原标注行数： 87473
标注中的唯一 stem 数量： 83920

X:\数据集\OceanShip_2\Oceanship_train 下共找到 68399 个 wav 文件，68399 个唯一 stem。

[Train 检查结果]
标注期望的 stem 数量: 83920
实际 wav stem 数量  : 68399
缺失的 wav 数量     : 15521
多余的 wav 数量     : 0

前 10 个缺失的 stem： ['20200904T032320.599Z_368_id_24_typecargo_31', '20200904T040544.726Z_452_id_5_typecargo_31', '20200904T042341.899Z_482_id_5_typecargo_31_0', '20200904T042341.899Z_482_id_5_typecargo_31_1', '20200904T045540.750Z_546_id_5_typecargo_31', '20200904T045942.590Z_550_id_5_typecargo_31_0', '20200904T045942.590Z_550_id_5_typecargo_31_1', '20200904T050227.843Z_560_id_5_typecargo_31_0', '20200904T050227.843Z_560_id_5_typecargo_31_1', '20200904T050227.843Z_560_id_5_typecargo_31_2']
前 10 个多余的 stem： []


# Test文件夹标注去重处理

In [15]:
# 标注文件路径
META_CSV = Path(r"X:\数据集\OceanShip_2\oceanship_full_test.csv")

# wav 所在根目录
WAV_ROOT = Path(r"X:\数据集\OceanShip_2\Oceanship_test")

# 标注里表示音频路径的列名
WAV_COL_NAME = "wav_path"

# 读取标注文件
df = pd.read_csv(META_CSV)
print("标注列名：", list(df.columns))
assert WAV_COL_NAME in df.columns, f"列 {WAV_COL_NAME} 不在标注文件中！"

# 抽取 stem（去掉目录和后缀，只保留主文件名）
from pathlib import Path

expected_stems = df[WAV_COL_NAME].astype(str).apply(lambda p: Path(p).stem)
expected_stem_set = set(expected_stems.tolist())

print(f"标注中共有 {len(df)} 条记录，"
      f"{len(expected_stem_set)} 个唯一 stem。")

# 扫描目录下所有 .wav
wav_paths = list(WAV_ROOT.rglob("*.wav"))
actual_stem_set = {p.stem for p in wav_paths}

print(f"{WAV_ROOT} 下共找到 {len(wav_paths)} 个 wav 文件，"
      f"{len(actual_stem_set)} 个唯一 stem。")

# 计算差集
missing_stems = sorted(expected_stem_set - actual_stem_set)  # 标注有，目录没有
extra_stems   = sorted(actual_stem_set - expected_stem_set)  # 目录有，标注没有

print(f"\n[检查结果]")
print(f"标注期望的 stem 数量: {len(expected_stem_set)}")
print(f"实际 wav stem 数量: {len(actual_stem_set)}")
print(f"缺失的 wav 数量: {len(missing_stems)}")
print(f"多余的 wav 数量: {len(extra_stems)}")

print("\n前 10 个缺失的 stem：", missing_stems[:10])
print("前 10 个多余的 stem：", extra_stems[:10])

标注列名： ['wav_path', 'label', 'text_path']
标注中共有 15450 条记录，15155 个唯一 stem。
X:\数据集\OceanShip_2\Oceanship_test 下共找到 15155 个 wav 文件，15155 个唯一 stem。

[检查结果]
标注期望的 stem 数量: 15155
实际 wav stem 数量: 15155
缺失的 wav 数量: 0
多余的 wav 数量: 0

前 10 个缺失的 stem： []
前 10 个多余的 stem： []


In [16]:
META_CSV = Path(r"X:\数据集\OceanShip_2\oceanship_full_test.csv")
df = pd.read_csv(META_CSV)

# 取 stem
df["stem"] = df["wav_path"].astype(str).apply(lambda p: Path(p).stem)

# 统计每个 stem 出现次数
counts = df["stem"].value_counts()

print("总行数:", len(df))
print("唯一 stem 数:", counts.shape[0])
print("出现超过 1 次的 stem 数量:", (counts > 1).sum())

# 看看前几个重复的例子
dup_examples = counts[counts > 1].head(5).index.tolist()
print("\n示例：这些 stem 在标注中出现了多次：", dup_examples)

df_dup = df[df["stem"].isin(dup_examples)]
df_dup.head(10)

总行数: 15450
唯一 stem 数: 15155
出现超过 1 次的 stem 数量: 295

示例：这些 stem 在标注中出现了多次： ['20210201T000447.782Z_12_id_5_typecargo_60_0', '20210131T055900.157Z_795_id_5_typecargo_31', '20210202T112011.517Z_1827_id_5_typecargo_53', '20210202T112133.917Z_1829_id_5_typecargo_31_0', '20210202T112602.957Z_1842_id_5_typecargo_51']


,wav_path,label,text_path,stem
6267,./v100_preprocessed_89_09_31/20210131T055900.1...,Towing,./v100_preprocessed_89_09_31/20210131T055900.1...,20210131T055900.157Z_795_id_5_typecargo_31
6625,./v100_preprocessed_89_09_31/20210201T000447.7...,Passenger,./v100_preprocessed_89_09_31/20210201T000447.7...,20210201T000447.782Z_12_id_5_typecargo_60_0
7280,./v100_preprocessed_89_09_31/20210202T112011.5...,Port Tender,./v100_preprocessed_89_09_31/20210202T112011.5...,20210202T112011.517Z_1827_id_5_typecargo_53
7281,./v100_preprocessed_89_09_31/20210202T112133.9...,Towing,./v100_preprocessed_89_09_31/20210202T112133.9...,20210202T112133.917Z_1829_id_5_typecargo_31_0
7282,./v100_preprocessed_89_09_31/20210202T112602.9...,Search and Rescue vessel,./v100_preprocessed_89_09_31/20210202T112602.9...,20210202T112602.957Z_1842_id_5_typecargo_51
14351,./3090latest_wav/20210131T055900.157Z_795_id_5...,Towing,./3090latest_wav/20210131T055900.157Z_795_id_5...,20210131T055900.157Z_795_id_5_typecargo_31
14875,./3090latest_wav/20210201T000447.782Z_12_id_5_...,Passenger,./3090latest_wav/20210201T000447.782Z_12_id_5_...,20210201T000447.782Z_12_id_5_typecargo_60_0
15411,./3090latest_wav/20210202T112011.517Z_1827_id_...,Port Tender,./3090latest_wav/20210202T112011.517Z_1827_id_...,20210202T112011.517Z_1827_id_5_typecargo_53
15413,./3090latest_wav/20210202T112133.917Z_1829_id_...,Towing,./3090latest_wav/20210202T112133.917Z_1829_id_...,20210202T112133.917Z_1829_id_5_typecargo_31_0
15414,./3090latest_wav/20210202T112602.957Z_1842_id_...,Search and Rescue vessel,./3090latest_wav/20210202T112602.957Z_1842_id_...,20210202T112602.957Z_1842_id_5_typecargo_51


In [18]:
# 2. 读取标注，增加 stem 列（去掉目录和后缀）
df = pd.read_csv(META_CSV)
print("原标注列名：", list(df.columns))
print("原标注行数：", len(df))

df["stem"] = df["wav_path"].astype(str).apply(lambda p: Path(p).stem)

# 3. 扫描实际存在的 wav，得到合法 stem 集合
wav_paths = list(WAV_ROOT.rglob("*.wav"))
wav_stems = {p.stem for p in wav_paths}
print("实际 wav 数量：", len(wav_paths))
print("实际唯一 stem 数量：", len(wav_stems))

# 4. 只保留“确实有对应 wav”的标注行（一般都会全部保留，这一步是安全检查）
df_valid = df[df["stem"].isin(wav_stems)].copy()
print("与 wav 匹配后的标注行数：", len(df_valid))

# 5. 按 stem 去重：同一个 wav 只保留一行标注
#    这里 keep='first' 表示保留第一次出现的那一条，你也可以改成 'last'
df_dedup = (
    df_valid
    .sort_values("stem")             # 先按 stem 排一下
    .drop_duplicates("stem", keep="first")
)

print("去重后的标注行数：", len(df_dedup))

# 6. 去掉临时的 stem 列，保存为新的 csv
out_csv = META_CSV.with_name(META_CSV.stem + "_dedup.csv")
df_dedup.drop(columns=["stem"]).to_csv(out_csv, index=False, encoding="utf-8-sig")

print(f"已保存去重后的标注文件到：{out_csv}")

原标注列名： ['wav_path', 'label', 'text_path']
原标注行数： 15450
实际 wav 数量： 15155
实际唯一 stem 数量： 15155
与 wav 匹配后的标注行数： 15450
去重后的标注行数： 15155
已保存去重后的标注文件到：X:\数据集\OceanShip_2\oceanship_full_test_dedup.csv


In [ ]:

# 1. 配置你的 test 音频目录
WAV_ROOT = Path(r"X:\数据集\OceanShip_2\Oceanship_test")

# 2. 收集所有 wav 文件
wav_paths = sorted(WAV_ROOT.rglob("*.wav"))
print(f"在 {WAV_ROOT} 下共找到 {len(wav_paths)} 个 wav 文件.")

if len(wav_paths) == 0:
    raise RuntimeError("没有找到 wav 文件，请检查 WAV_ROOT 路径是否正确。")

# 3. 随机抽取 10 个（如果总数小于 10，就全测）
N = 10
if len(wav_paths) <= N:
    sample_paths = wav_paths
else:
    sample_paths = random.sample(wav_paths, N)

print(f"\n准备检查 {len(sample_paths)} 个文件的采样率、通道数和时长：\n")

# 4. 逐个读取并打印信息
for i, p in enumerate(sample_paths, start=1):
    data, sr = sf.read(p)  # data: numpy array, sr: sample rate
    # 通道数
    if data.ndim == 1:
        channels = 1
        num_samples = data.shape[0]
    else:
        channels = data.shape[1]
        num_samples = data.shape[0]

    duration_sec = num_samples / sr

    print(f"[{i}] {p.name}")
    print(f"    采样率 (sample rate): {sr} Hz")
    print(f"    通道数 (channels)   : {channels}")
    print(f"    样本点数 (samples)  : {num_samples}")
    print(f"    时长 (duration)     : {duration_sec:.3f} 秒")
    print(f"    数据类型 (dtype)    : {data.dtype}")
    print("-" * 60)

在 X:\数据集\OceanShip_2\Oceanship_test 下共找到 15155 个 wav 文件.

准备检查 10 个文件的采样率、通道数和时长：

[1] 20210106T010452.131Z_134_id_5_typecargo_60_0.wav
    采样率 (sample rate): 32000 Hz
    通道数 (channels)   : 1
    样本点数 (samples)  : 160000
    时长 (duration)     : 5.000 秒
    数据类型 (dtype)    : float64
------------------------------------------------------------
[2] 20201205T033704.303Z_437_id_5_typecargo_52_1.wav
    采样率 (sample rate): 32000 Hz
    通道数 (channels)   : 1
    样本点数 (samples)  : 160000
    时长 (duration)     : 5.000 秒
    数据类型 (dtype)    : float64
------------------------------------------------------------
[3] 20200814T000035.853Z_1_id_5_typecargo_52_2.wav
    采样率 (sample rate): 32000 Hz
    通道数 (channels)   : 1
    样本点数 (samples)  : 76416
    时长 (duration)     : 2.388 秒
    数据类型 (dtype)    : float64
------------------------------------------------------------
[4] 20201028T055212.450Z_693_id_5_typecargo_31_1.wav
    采样率 (sample rate): 32000 Hz
    通道数 (channels)   : 1
    样本点数 (samples)  : 16

In [6]:
TARGET_SR = 32000
TARGET_DUR = 5.0
TARGET_LEN = int(TARGET_SR * TARGET_DUR) 

lengths = []
for p in WAV_ROOT.rglob("*.wav"):
    data, sr = sf.read(str(p))
    if data.ndim > 1:
        data = data.mean(axis=1)
    lengths.append(len(data))

lengths = np.array(lengths)
short = (lengths < TARGET_LEN)
print("总文件数:", len(lengths))
print("不足 5s 的数量:", short.sum())
print("不足 5s 的比例: {:.2f}%".format(short.mean() * 100))

总文件数: 15155
不足 5s 的数量: 8370
不足 5s 的比例: 55.23%
